In [1]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [2]:
import pke
import string
from nltk.corpus import stopwords

from googlesearch import search
import requests
import textdistance

In [3]:
extractor = pke.unsupervised.TopicRank()

# TODO: Process closed-caption stream and extract keywords for every 5 sentences
extractor.load_document(input='mushroom.txt')

pos = {'NOUN', 'PROPN', 'ADJ'}
stoplist = list(string.punctuation)
stoplist += ['-lrb-', '-rrb-', '-lcb-', '-rcb-', '-lsb-', '-rsb-']
stoplist += stopwords.words('english')
extractor.candidate_selection(pos=pos, stoplist=stoplist)

extractor.candidate_weighting(threshold=0.74, method='average')

keyphrases = extractor.get_n_best(n=5, redundancy_removal=True)
keyphrases = [keyphrase for (keyphrase, weight) in keyphrases]
for keyphrase in keyphrases:
    print(keyphrase)

mycelium
material
biotech startup ecovative
company
natural alternative


In [4]:
wiki_query = ' '.join(keyphrases + ['wikipedia'])

for result in search(wiki_query, tld='com', lang='en', num=10, stop=10, pause=2):
    if 'wikipedia.org' in result:
        title = result.split('/')[-1]
        wiki_request = 'https://en.wikipedia.org/w/api.php?action=opensearch&search=' + title + '&limit=1&namespace=0&format=json'
        wiki = requests.get(wiki_request, timeout=(5, 20))
        wiki_card = wiki.json()
        break

In [5]:
relevant_articles = []
news_query = '%20'.join(keyphrases).replace(' ', '%20')
while len(relevant_articles) < 8:
    news_request = 'https://newsapi.org/v2/everything?q=' + news_query + '&sortBy=relevancy&pageSize=10&apiKey=f086208033f04214bd283cc403681fc1'
    news = requests.get(news_request, timeout=(5, 20))
    news_list = news.json()['articles']
    for article in news_list:
        title = article['title']
        url = article['url']
        date = article['publishedAt']
        is_duplicate = False
        for t, _, _ in relevant_articles:
            if textdistance.levenshtein.normalized_distance(title, t) < 0.5:
                is_duplicate = True
                break
        if not is_duplicate:
            relevant_articles.append((title, url, date))
            if len(relevant_articles) == 8:
                break
    news_query = news_query.replace('%20' + news_query.split('%20')[-1], '')

In [6]:
print("Wiki card:")
print("Title: " + wiki_card[0])
print("Summary: " + wiki_card[2][0])
print("Url: " + wiki_card[3][0])
print()

print("Relevant news articles:")
for i, (title, url, date) in enumerate(relevant_articles):
    print(str(i + 1))
    print("Title: " + title)
    print("Url: " + url)
    print("Date:" + date)
    print()

Wiki card:
Title: Ecovative_Design
Summary: Ecovative Design LLC is a biomaterials company headquartered in Green Island, New York that provides sustainable alternatives to plastics and polystyrene foams for packaging, building materials and other applications by using mushroom technology.
Url: https://en.wikipedia.org/wiki/Ecovative_Design

Relevant news articles:
1
Title: This Company Is Using Mushrooms To Reduce Plastic Waste
Url: https://science.slashdot.org/story/19/11/17/2248250/this-company-is-using-mushrooms-to-reduce-plastic-waste
Date:2019-11-17T22:50:00Z

2
Title: Mushrooms may help reduce plastic waste
Url: https://www.koamnewsnow.com/news/national-news/mushrooms-may-help-reduce-plastic-waste/1142016711
Date:2019-11-14T07:00:19Z

3
Title: The Weirdness Is Coming
Url: http://nymag.com/intelligencer/2019/11/2029-predictions-based-on-2019.html
Date:2019-11-13T11:00:20Z

4
Title: Mushrooms, Cattail Reeds, and Agricultural Waste are Reimagined to Construct “The Growing Pavilion”